# O problema

Gerar relatórios iniciais a serem apresentados em uma reunião de investidores e acionistas, explicando a quantidade de vinhos exportados e os fatores externos que podem vir a surgir e que interferem nas análises:

1. Dados climáticos.
2. Dados demográficos.
3. Dados econômicos.
4. Dados de avaliações de vinhos.

O head de dados pediram para que você construísse uma tabela contendo as seguintes informações:

1. País de origem (Brasil).
2. País de destino.
3. Quantidade em litros de vinho exportado (utilize: 1kg = 1l).
4. Valor em US$.

Seu objetivo é dizer o montante de venda de exportação nos últimos 15 anos, separando a análise por país e trazendo quais as propecções futuras e possíveis ações para uma melhoria nas exportações. Construa gráficos atraentes e que passem a ideia central para que os acionistas e investidores possam seguir em frente com suas ações.

fontes: 

http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_06

https://pt.wikipedia.org/wiki/Lista_de_pa%C3%ADses_por_popula%C3%A7%C3%A3o

## Importando bibliotecas

In [1]:
import pandas as pd

import warnings 
warnings.filterwarnings(action = 'ignore')

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import plotly.express as px

## Importando dados

In [2]:
dados = pd.read_csv('../dados_uteis/dados.csv')

In [3]:
dados.head()

,ano,pais,quantidade_exportacao,valor_exportacao,valor_exportacao_por_litro,producao_brasileira,percentual_exportacao
0,2007,Afeganistao,0,0,0.00,318464392,0.000000
1,2007,Africa do Sul,0,0,0.00,318464392,0.000000
2,2007,Alemanha,119512,238052,1.99,318464392,0.037528
3,2007,Angola,25931,49753,1.92,318464392,0.008143
4,2007,Anguilla,0,0,0.00,318464392,0.000000


## Análise exploratória

### Exportação

#### Montante dos valores e quantidades na exportação negociada com os principais mercados últimos 15 anos

In [4]:
acumulado_top_10 = pd.DataFrame(dados.groupby('pais').sum()[['quantidade_exportacao', 'valor_exportacao']])
acumulado_top_10.head()

,quantidade_exportacao,valor_exportacao
pais,,
Afeganistao,11,46
Africa do Sul,30,116
Alemanha,1020933,2739079
Angola,190731,621181
Anguilla,0,0


In [5]:
selecao = acumulado_top_10.sort_values('valor_exportacao', ascending = False).head(10).index #selecionando os 10 maiores mercados de exportação
selecao

Index(['Paraguai', 'Russia', 'Estados Unidos', 'Reino Unido', 'China',
       'Holanda', 'Espanha', 'Alemanha', 'Japao', 'Haiti'],
      dtype='object', name='pais')

In [6]:
acumulado_top_10 = acumulado_top_10.loc[selecao]
acumulado_top_10.reset_index(inplace = True)
acumulado_top_10.head()

,pais,quantidade_exportacao,valor_exportacao
0,Paraguai,25423559,32364257
1,Russia,39230153,25559026
2,Estados Unidos,3822251,10046712
3,Reino Unido,1305329,4717554
4,China,2411701,4372268


In [7]:
fig = px.bar(
    data_frame = acumulado_top_10,
    x = 'pais',
    y = ['quantidade_exportacao', 'valor_exportacao'],
    barmode = 'group', 
    color_discrete_map = {
        'quantidade_exportacao': '#673E69',
        'valor_exportacao': '#D8D87C'
        }
    )

nome_variaveis = {'quantidade_exportacao': 'Quantidade (Litros)', 'valor_exportacao': 'Valor (US$)'}
fig.for_each_trace(lambda x: x.update(
                                    name = nome_variaveis[x.name],
                                    legendgroup = nome_variaveis[x.name],
                                    hovertemplate = x.hovertemplate.replace(x.name, nome_variaveis[x.name])
                                )
    )

fig.update_layout(
    title = 'Montante da Exportação de Vinho Brasileiro nos Último 15 anos', 
    xaxis_title = 'Países', 
    yaxis_title = 'Valores', 
    legend_title = 'Legenda',
    width = 1000, 
    height = 600, 
    bargap = 0.1
    )

fig.show()

In [8]:
fig = px.line(
    data_frame = acumulado_top_10, 
    x = acumulado_top_10.pais, 
    y = acumulado_top_10.quantidade_exportacao, 
    color = px.Constant('Quantidade (Litro)'), 
    color_discrete_sequence = ['#673E69']
    )

fig.add_bar(
    x = acumulado_top_10.pais, 
    y = acumulado_top_10.valor_exportacao, 
    name = 'Valor (US$)',
    marker = dict(color= '#D8D87C')
    )

fig.update_layout(
    title = 'Montante da Exportação de Vinho Brasileiro nos Último 15 anos', 
    xaxis_title = 'Países', 
    yaxis_title = 'Valores', 
    legend_title = 'Legenda', 
    width = 1000, 
    height = 600)

fig.show()

In [9]:
fig = px.scatter(
    data_frame = acumulado_top_10, 
    x = acumulado_top_10.quantidade_exportacao, 
    y = acumulado_top_10.valor_exportacao,  
    hover_data = 'pais',
    color_discrete_sequence = ['#673E69']
    )

fig.update_layout(
    title = 'Montante da Exportação de Vinho Brasileiro nos Último 15 anos', 
    xaxis_title = 'Quantidade (Litros)', 
    yaxis_title = 'Valor (US$)', 
    width = 600, 
    height = 600)

fig.update_traces(marker = dict(size = 8))

fig.show()

Conclusões:

Vê-se que, nos últimos 15 anos, o comércio com a Rússia e o Haiti apresentaram os piores resultados no acumulado. 

O comércio com o Paraguai apresenta o maior resultado absoluto.

#### Total de quantidade exportada nos últimos 15 anos

In [10]:
dados.quantidade_exportacao.sum()

84238201

#### Total de valores negociados nos últimos 15 anos

In [11]:
dados.valor_exportacao.sum()

105385184

#### Evolução da quantidade e valores negociados de exportação nos últimos 15 anos

In [12]:
evolucao_exportacao = dados.groupby('ano').sum().drop('pais', axis = 1)
evolucao_exportacao.head()

,quantidade_exportacao,valor_exportacao,valor_exportacao_por_litro,producao_brasileira,percentual_exportacao
ano,,,,,
2007,3281752,3686150,83.55,39489584608,1.030493
2008,10346323,7118100,151.00,41520322688,3.089918
2009,25514198,8940988,121.79,30419527976,10.400426
2010,1280574,2595303,140.30,27289137932,0.581884
2011,1214834,3615120,179.91,37874463280,0.397733


In [13]:
fig = px.line(
    evolucao_exportacao, 
    x = evolucao_exportacao.index, 
    y = ['valor_exportacao', 'quantidade_exportacao'],
    color_discrete_map = {
        'quantidade_exportacao': '#673E69',
        'valor_exportacao': '#D8D87C'
        }
)

nome_variaveis = {'quantidade_exportacao': 'Quantidade (Litros)', 'valor_exportacao': 'Valor (US$)'}
fig.for_each_trace(lambda x: x.update(
                                    name = nome_variaveis[x.name],
                                    legendgroup = nome_variaveis[x.name],
                                    hovertemplate = x.hovertemplate.replace(x.name, nome_variaveis[x.name])
                                )
    )

fig.update_layout(
    title = 'Montante da Exportação de Vinho Brasileiro nos Últimos 15 anos', 
    xaxis_title = 'Países', 
    yaxis_title = 'Valores', 
    legend_title = 'Legenda', 
    width = 1000, 
    height = 600
    )

fig.show()

Conclusões:

Em 2009, o mercado de vinho brasileiro apresentou rentabilidade muito menor, no que diz respeito a exportação, que nos anos subsequentes.

Já em 2013, houve o inverso, os valores negociados foram substanciamente maiores que em outros anos da série.

Essa dinâmica foi fortemente influenciada pela Rússia.

In [14]:
dados.query('quantidade_exportacao > 10000000')

,ano,pais,quantidade_exportacao,valor_exportacao,valor_exportacao_por_litro,producao_brasileira,percentual_exportacao
349,2009,Russia,21912914,5732280,0.26,245318774,8.932424


In [15]:
dados.query('valor_exportacao > 10000000')

,ano,pais,quantidade_exportacao,valor_exportacao,valor_exportacao_por_litro,producao_brasileira,percentual_exportacao
845,2013,Russia,5893291,14795694,2.51,242686752,2.428353


In [16]:
fig = px.line(
    dados.query('pais == "Russia"'), 
    x = 'ano', 
    y = ['valor_exportacao', 'quantidade_exportacao'],
    color_discrete_map = {
        'quantidade_exportacao': '#673E69',
        'valor_exportacao': '#D8D87C'
        }
)

nome_variaveis = {'quantidade_exportacao': 'Quantidade (Litros)', 'valor_exportacao': 'Valor (US$)'}
fig.for_each_trace(lambda x: x.update(
                                    name = nome_variaveis[x.name],
                                    legendgroup = nome_variaveis[x.name],
                                    hovertemplate = x.hovertemplate.replace(x.name, nome_variaveis[x.name])
                                )
    )

fig.update_layout(
    title = 'Evolução da Exportação de Vinho Brasileiro para a Rússia', 
    xaxis_title = 'Período', 
    yaxis_title = 'Valores', 
    legend_title = 'Legenda', 
    width = 1000, 
    height = 600
    )

fig.show()

Nota-se que a Rússia, apesar de em números absolutos ter uma grande parcela na exportação, não é um mercado regular.

#### Evolução da exportação para os principais mercados nos últimos 15 anos

In [17]:
selecao = acumulado_top_10['pais'].to_list()
selecao

['Paraguai',
 'Russia',
 'Estados Unidos',
 'Reino Unido',
 'China',
 'Holanda',
 'Espanha',
 'Alemanha',
 'Japao',
 'Haiti']

In [18]:
evolucao_top_10 = dados[dados.pais.isin(selecao)]
evolucao_top_10.head()

,ano,pais,quantidade_exportacao,valor_exportacao,valor_exportacao_por_litro,producao_brasileira,percentual_exportacao
2,2007,Alemanha,119512,238052,1.99,318464392,0.037528
28,2007,China,7638,30390,3.98,318464392,0.002398
46,2007,Espanha,0,0,0.00,318464392,0.000000
47,2007,Estados Unidos,479269,810038,1.69,318464392,0.150494
60,2007,Haiti,0,0,0.00,318464392,0.000000


In [19]:
fig = px.line(
    data_frame = evolucao_top_10, 
    x = 'ano', 
    y = 'valor_exportacao',
    color = 'pais'
)

fig.update_layout(
    title = 'Evolução da Exportação de Vinho Brasileiro', 
    xaxis_title = 'Período', 
    yaxis_title = 'Valor (US$)', 
    legend_title = 'Legenda',
    width = 1000, 
    height = 600
    )

fig.show()

In [20]:
fig = px.line(
    data_frame = evolucao_top_10, 
    x = 'ano', 
    y = 'quantidade_exportacao',
    color = 'pais'
)

fig.update_layout(
    title = 'Evolução da Exportação de Vinho Brasileiro', 
    xaxis_title = 'Período', 
    yaxis_title = 'Quantidade (Litros)', 
    legend_title = 'Legenda',
    width = 1000, 
    height = 600
    )

fig.show()

Observa-se que o Paraguai é o único mercado importador que apresenta um crescimento estável desde 2015.

#### Preço por litro

In [21]:
print(f'Preço médio: {dados["valor_exportacao_por_litro"].mean():.2f}') #preço médio por litro nos últimos 15 anos

Preço médio: 1.50


In [22]:
evolucao_preco_medio_por_litro = dados.groupby('ano').mean('valor_exportacao_por_litro').round(2)
evolucao_preco_medio_por_litro.head()

,quantidade_exportacao,valor_exportacao,valor_exportacao_por_litro,producao_brasileira,percentual_exportacao
ano,,,,,
2007,26465.74,29727.02,0.67,318464392.0,0.01
2008,83438.09,57404.03,1.22,334841312.0,0.02
2009,205759.66,72104.74,0.98,245318774.0,0.08
2010,10327.21,20929.86,1.13,220073693.0,0.00
2011,9797.05,29154.19,1.45,305439220.0,0.00


In [23]:
fig = px.line(
    data_frame = evolucao_preco_medio_por_litro, 
    x = evolucao_preco_medio_por_litro.index, 
    y = 'valor_exportacao_por_litro',
    color_discrete_sequence = ['#673E69']
)

fig.update_layout(
    title = 'Evolução do Preço Médio por Litro nos Últimos 15 Anos', 
    xaxis_title = 'Período', 
    yaxis_title = 'Valor (US$)',
    width = 1000, 
    height = 600
    )

fig.show()

In [24]:
preco_medio_por_litro_top_10 = evolucao_top_10[['pais', 'valor_exportacao_por_litro']].groupby('pais').mean().round(2)
preco_medio_por_litro_top_10.head()

,valor_exportacao_por_litro
pais,
Alemanha,4.18
China,4.28
Espanha,1.29
Estados Unidos,2.99
Haiti,0.63


In [25]:
fig = px.bar(
    data_frame = preco_medio_por_litro_top_10, 
    x = preco_medio_por_litro_top_10.index, 
    y = 'valor_exportacao_por_litro',
    color_discrete_sequence = ['#D8D87C']
)

fig.update_layout(
    title = 'Preço Médio por Litro nos Últimos 15 Anos', 
    xaxis_title = 'Período', 
    yaxis_title = 'Valor (US$)',
    width = 1000, 
    height = 600
    )

fig.show()

In [26]:
fig3 = px.treemap(preco_medio_por_litro_top_10, path = [preco_medio_por_litro_top_10.index], values = 'valor_exportacao_por_litro', title = 'Preço Médio por Litro nos Últimos 15 Anos', width = 1000, height = 600)
fig3.update_traces(root_color = '#D8D87C')

In [27]:
fig4 = px.line(
    data_frame = evolucao_top_10, 
    x = 'ano', 
    y = 'valor_exportacao_por_litro',
    color = 'pais'
    )
fig4.update_layout(
    title = 'Evolução do Preço por Litro para os 10 Principais Importadores', 
    xaxis_title = 'Período', 
    yaxis_title = 'Valor (US$)', 
    legend_title = 'Legenda',
    width = 1000,
    height = 600
    )

Conclusões:

Entre os 10 maiores mercados de exportação do vinho brasileiro, Espanha, Rússia e Haiti apresentaram períodos de interrupções nos últimos 15 anos.

Mercados menos significativos em números absolutos apresentaram maior rentabilidade. 

O Paraguai foi o mercado importador cujos preços negociados apresentaram maior estabilidade nos últimos 15 anos. 

Apesar do grande volume exportado para a Rússia, e preço médio por litro está acima apenas do Haiti, último colocado entre os 10.

#### Percentual Exportado da Produção Nacional

In [28]:
print(f'Percentual médio: {dados["percentual_exportacao"].mean():.3f}') #percentual médio

Percentual médio: 0.019


In [29]:
percentual_exportacao = dados[['ano', 'pais', 'percentual_exportacao']].round(2)
percentual_exportacao.head()

,ano,pais,percentual_exportacao
0,2007,Afeganistao,0.00
1,2007,Africa do Sul,0.00
2,2007,Alemanha,0.04
3,2007,Angola,0.01
4,2007,Anguilla,0.00


In [30]:
evolucao_percentual_medio_exportacao = percentual_exportacao.groupby('ano').mean('percentual_exportacao').round(2)
evolucao_percentual_medio_exportacao.head()

,percentual_exportacao
ano,
2007,0.01
2008,0.02
2009,0.08
2010,0.00
2011,0.00


In [31]:
fig = px.line(
    data_frame = evolucao_percentual_medio_exportacao, 
    x = evolucao_percentual_medio_exportacao.index, 
    y = 'percentual_exportacao',
    color_discrete_sequence = ['#673E69']
)

fig.update_layout(
    title = 'Evolução do Percentual Médio Exportado da Produção Nacional nos Últimos 15 Anos', 
    xaxis_title = 'Período', 
    yaxis_title = '%',
    width = 1000, 
    height = 600
    )

fig.show()

In [32]:
percentual_medio_exportacao_top_10 = evolucao_top_10.groupby('pais').mean('percentual_exportacao').round(2)
percentual_medio_exportacao_top_10.head()

,ano,quantidade_exportacao,valor_exportacao,valor_exportacao_por_litro,producao_brasileira,percentual_exportacao
pais,,,,,,
Alemanha,2014.0,68062.20,182605.27,4.18,241617313.0,0.03
China,2014.0,160780.07,291484.53,4.28,241617313.0,0.07
Espanha,2014.0,132866.67,253903.47,1.29,241617313.0,0.05
Estados Unidos,2014.0,254816.73,669780.80,2.99,241617313.0,0.11
Haiti,2014.0,82540.00,105746.27,0.63,241617313.0,0.04


In [33]:
fig = px.bar(
    data_frame = percentual_medio_exportacao_top_10, 
    x = percentual_medio_exportacao_top_10.index, 
    y = 'percentual_exportacao',
    color_discrete_sequence = ['#D8D87C']
)

fig.update_layout(
    title = 'Percentual Médio Exportado da Produção Nacional nos Últimos 15 Anos', 
    xaxis_title = 'Países', 
    yaxis_title = '%', 
    width = 1000, 
    height = 600
    )

fig.show()

In [34]:
percentual_medio_exportacao_top_10
fig = px.pie(percentual_medio_exportacao_top_10, values = 'percentual_exportacao', names = percentual_medio_exportacao_top_10.index, title = 'Percentual Médio da Produção Nacional para os Principais Importadores')
fig.show()

In [35]:
fig7 = px.treemap(percentual_medio_exportacao_top_10, path = [percentual_medio_exportacao_top_10.index], values = 'percentual_exportacao', title = 'Percentual Médio Exportado da Produção Nacional nos Últimos 15 Anos',  width = 1000, height = 600)
fig7.update_traces(root_color = '#D8D87C')

In [36]:
fig4 = px.line(
    data_frame = evolucao_top_10, 
    x = 'ano', 
    y = 'percentual_exportacao',
    color = 'pais'
    )
fig4.update_layout(
    title = 'Evolução do Percentual Médio para os 10 Principais Importadores', 
    xaxis_title = 'Período', 
    yaxis_title = 'Valor (US$)', 
    legend_title = 'Legenda',
    width = 1000,
    height = 600
    )

Conclusões:

O Paraguai e a Rússia são os países importadores com maior participação na exportação da produção nacional.

#### Distribuição da Exportação

In [37]:
index = evolucao_top_10.query('quantidade_exportacao == 0').index.to_list()
distribuicao_top_10 = evolucao_top_10.drop(index)
distribuicao_top_10.head()

,ano,pais,quantidade_exportacao,valor_exportacao,valor_exportacao_por_litro,producao_brasileira,percentual_exportacao
2,2007,Alemanha,119512,238052,1.99,318464392,0.037528
28,2007,China,7638,30390,3.98,318464392,0.002398
47,2007,Estados Unidos,479269,810038,1.69,318464392,0.150494
73,2007,Japao,357943,316867,0.89,318464392,0.112397
91,2007,Holanda,181046,364751,2.01,318464392,0.056850


In [38]:
fig = px.scatter(
    data_frame = distribuicao_top_10, 
    x = 'pais', 
    y = 'ano', 
    size = 'valor_exportacao', 
    color = px.Constant('Valor (US$)'),
    color_discrete_sequence = ['#673E69'])
fig.update_layout(
    title = 'Distribuição do Valor de Exportação para os Principais Importadores', 
    xaxis_title = 'Países', 
    yaxis_title = 'Ano', 
    legend_title = 'Legenda',
    width = 1000,
    height = 800
    )
fig.update_traces(marker = dict(size = 3.5 * distribuicao_top_10['valor_exportacao']))

In [39]:
fig = px.scatter(
    data_frame = distribuicao_top_10, 
    x = 'pais', 
    y = 'ano', 
    size = 'quantidade_exportacao', 
    color = px.Constant('Quantidade (Litros)'),
    color_discrete_sequence = ['#D8D87C'])
fig.update_layout(
    title = 'Distribuição da Quantidade de Exportação para os Principais Importadores', 
    xaxis_title = 'Países', 
    yaxis_title = 'Ano', 
    legend_title = 'Legenda',
    width = 1000,
    height = 800
    )
fig.update_traces(marker = dict(size = 3.5 * distribuicao_top_10['quantidade_exportacao']))